In [6]:
import cv2
import matplotlib.pyplot as plt 
import numpy as np 

In [7]:
import tensorflow as tf

In [3]:
import tflite_runtime.interpreter as tflite

ModuleNotFoundError: No module named 'tflite_runtime'

In [10]:
ls model 

ei_image_classification.py  trained.tflite
labels.txt


In [11]:
model_path = "model/trained.tflite"

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Allocate tensors
interpreter.allocate_tensors()

ValueError: Didn't find op for builtin opcode 'CONV_2D' version '3'
Registration failed.


In [8]:
image_transform = cv2.imread("1.bmp", cv2.IMREAD_GRAYSCALE)
print(image_transform)
# plt.imshow(image_transform)
# plt.show()

None


In [3]:
ls

1.bmp*
Img_1_473385748.bmp*
Img_24_473387381.bmp
Img_4_473386053.bmp*
Img_5_473386114.bmp*
Img_6_473386175.bmp*
Img_7_473386236.bmp*
Img_8_473386296.bmp*
ROI_0.png
ROI_1.png
ROI_2.png
ROI_3.png
ROI_4.png
ROI_5.png
Untitled.ipynb
crop_save_image.ipynb
cropped/
cropping_digit.ipynb
cropping_digit_with_perpective_transform.ipynb
draw_digit_with_perpective_transform_crop_save_image.ipynb
meter.png
rotational_correction.ipynb
test.png
testing.ipynb
transformed.jpg
transformed2.jpg
